In [5]:
import os
import sys
from bs4 import BeautifulSoup as bs
import pandas as pd
import math

cwd = r'D:\test_development\LCK_html\Main\WebApp'

In [6]:
def append_table(soup, building):
    df_file = pd.read_csv(cwd + '/source/data/' + building + '.csv')
    df_name = pd.read_csv(cwd + '/source/raw/rank.csv', index_col='Page')

    number = 0
    for name, df in list(df_file.groupby('Page')):
        for i in range(math.ceil(len(df)/5)):
            number += 1
            div_full = soup.new_tag('div', **{'class':'full'})
            table_df = df[i*5:i*5+5][['Shown Name', 'Value', 'Unit']]

            table_df = pd.concat([table_df, pd.DataFrame([['\\'] * table_df.shape[1]] * (5 - len(table_df)), columns=table_df.columns)])

            html_df = table_df.to_html(index=False, header=False, table_id='table_' + str(number))
            html_df = html_df.replace('border="1" ', '').replace('<td>\\', '<td id="hide">\\<br />\\').replace('\\n', '<br />')

            table = bs(html_df, 'html.parser')
            chinese, english = df_name['Name'][name].split('\n')
            thead = bs('<thead><tr><th colspan="3">' + chinese + '<br />' + english + '</th></tr></thead>', 'html.parser')
            table.find('tbody').insert_before(thead)

            div_full.append(table)
            soup.find('section', {'id':'tables'}).append(div_full)

In [15]:
code = 'B02'
blocks = pd.read_csv(cwd + '/source/raw/blocks.csv', index_col='Block No.')
# if code not in blocks.index:
#     return 'code not found'
print(blocks['Name'][code])

with open(cwd + '/templates/template.html', encoding="utf8") as template:
    soup = bs(template, 'html.parser')

append_table(soup, blocks['Name'][code])
# append_table(soup, blocks['Name']['B01']) #B00

chinese_estate, english_estate = blocks['Name']['B00'].split('_', 1)
estate = bs('<div>' + chinese_estate + '</div><div>' + english_estate + '</div>', 'html.parser')
soup.find('footer').append(estate)

chinese_building,  english_building = blocks['Name'][code].split('_', 1)
building = bs(chinese_building + '<br />' + english_building, 'html.parser')
soup.find('div', {'id':'building'}).append(building)

for file in os.listdir(cwd + '/static'):
    if file.endswith('.mp4'):
        video = bs('<video autoplay="" muted="" src="../files/' + file + '"></video>', 'html.parser')
        soup.find('section', {'id':'videos'}).append(video)
    if file.endswith('.jpg'):
        logo = bs('<img id="logo" src="../files/' + file + '" />', 'html.parser')
        soup.find('div', {'id':'system'}).append(logo)

print(soup.prettify())

黃蝶樓_Wong Tip House
<!DOCTYPE html>
<html>
 <head>
  <meta charset="utf-8" content="" http-equiv="refresh"/>
  <link href="../files/style.css" rel="stylesheet" type="text/css"/>
  <script src="../files/jquery.min.js" type="text/javascript">
  </script>
 </head>
 <body>
  <main>
   <aside>
    <div class="seg flex_center display" id="system">
     <div>
      能源監控系統
      <br/>
      Energy Monitoring System
     </div>
     <img id="logo" src="../files/hkha_logo.jpg"/>
     <img id="logo" src="../files/hkha_logo.jpg"/>
    </div>
    <div class="seg flex_center" id="building">
     黃蝶樓
     <br/>
     Wong Tip House
    </div>
    <div class="seg flex_center" id="time">
    </div>
   </aside>
   <article>
    <section class="full display" id="tables">
     <div class="full">
      <table class="dataframe" id="table_1">
       <thead>
        <tr>
         <th colspan="3">
          住戶用電量
          <br/>
          Tenant Energy Consumption
         </th>
        </tr>
       </thead>
   